In [ ]:
import cv2
import numpy as np
import time
from random import randint


#######################
# v2版，更新：
# 1、增加四条车道划线；
# 2、增加四条车道的点数。
#######################

def isInsidePolygon(pt, poly):
    c = False
    i = -1
    l = len(poly)
    j = l - 1
    while i < l - 1:
        i += 1
        if ((poly[i][0] <= pt[0] and pt[0] < poly[j][0])
                or (poly[j][0] <= pt[0] and pt[0] < poly[i][0])):
            if (pt[1] < (poly[j][1] - poly[i][1])
                    * (pt[0] - poly[i][0]) / (poly[j][0] - poly[i][0])
                    + poly[i][1]):
                c = not c
        j = i
    return c


class Car:
    tracks = []  # 车辆坐标轨迹

    def __init__(self, i, xi, yi, max_age):
        self.i = i
        self.x = xi
        self.y = yi
        self.tracks = []  # 历史轨迹
        self.R = randint(0, 255)
        self.G = randint(0, 255)
        self.B = randint(0, 255)
        self.done = False  # 是否完成
        self.state = '0'  # 车辆过线识别
        self.age = 0
        self.max_age = max_age
        self.dir = None  # 车辆上下行状态

    def getRGB(self):  # For 随机生成颜色，用于不同车辆ID观察
        return (self.R, self.G, self.B)

    def getTracks(self):
        return self.tracks

    def getId(self):  # 车辆ID
        return self.i

    def getState(self):
        return self.state

    def getDir(self):
        return self.dir

    def getX(self):  # for x coordinate
        return self.x

    def getY(self):  # for y coordinate
        return self.y

    def updateCoords(self, xn, yn):
        self.age = 0
        self.tracks.append([self.x, self.y])
        self.x = xn
        self.y = yn

    def setDone(self):
        self.done = True

    def timedOut(self):
        return self.done

    def going_UP(self, mid_start, mid_end):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] < mid_end and self.tracks[-2][1] >= mid_end:
                    self.state = '1'
                    self.dir = 'up'
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False

    def going_DOWN(self, mid_start):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] > mid_start and self.tracks[-2][1] <= mid_start:
                    self.state = '1'
                    self.dir = 'down'
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False

    def age_one(self):
        self.age += 1
        if self.age > self.max_age:
            self.done = True
        return True


####################### 开始
cap = cv2.VideoCapture("2001.MOV")
# 预设
f = 0.5  # 缩放比例
r = True  # 是否旋转
rectangle_color = (0, 255, 0)  # 识别框颜色
cedao_pts_1 = np.array([[210, 110], [270, 110], [0, 333], [0, 245]], np.int32)
cedao_pts_2 = np.array([[274, 110], [335, 110], [73, 430], [0, 430], [0, 337]], np.int32)
cedao_pts_3 = np.array([[338, 110], [401, 110], [265, 430], [77, 430]], np.int32)
cedao_pts_4 = np.array([[403, 110], [465, 110], [465, 430], [267, 430]], np.int32)
# 上下方跟踪线
line_limit_up = 110
line_limit_down = 430
line_limit_left = 0
line_limit_right = 465
# Get width and height of video
w = cap.get(3) * f
h = cap.get(4) * f
frameArea = h * w
areaTH = frameArea / 450  # 识别轮廓，默认视频面积的400分之1
# 下行计数线
line_count_go_down = 245
line_count_go_down_color = (255, 255, 255)
# 化划线坐标
pt1 = [0, line_count_go_down]
pt2 = [w, line_count_go_down]
pts_L1 = np.array([pt1, pt2], np.int32)
pts_L1 = pts_L1.reshape((-1, 1, 2))  # 矩阵变化：转为个数未知的1行2列矩阵
# 背景分割：高斯混合背景分割算法
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
# 多维数组，用于形态学去噪
kernalOp = np.ones((3, 3), np.uint8)
kernalOp2 = np.ones((5, 5), np.uint8)
kernalCl = np.ones((11, 11), np.uint8)
# 初始化变量
font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
max_p_age = 5  # 车辆最大计数，超过则car.done=True
pid = 1
cnt_frame = 0
cnt_go_down_all = 0
cnt_go_down_1 = 0
cnt_go_down_2 = 0
cnt_go_down_3 = 0
cnt_go_down_4 = 0
# 循环处理每一帧并播放
while (cap.isOpened()):
    cnt_frame += 1
    ret, frame = cap.read()
    # 旋转帧
    if r == True:
        frame = cv2.flip(frame, -1)
    # 缩放帧
    if f != 1:
        frame = cv2.resize(frame, None, fx=f, fy=f, interpolation=cv2.INTER_CUBIC);
    # 每一帧车辆年龄+1
    for i in cars:
        i.age_one()
    # 获取前景遮罩
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)
    if ret == True:
        # 图像二值化去噪突显轮廓
        ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        ret, imBin2 = cv2.threshold(fgmask2, 200, 255, cv2.THRESH_BINARY)
        # 形态学去噪，開運算是先侵蝕後膨脹，閉運算是先膨脹後侵蝕。
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernalOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_CLOSE, kernalOp)
        # 形态学去噪
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernalCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernalCl)
        # 获取所有轮廓并循环处理
        countours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for cnt in countours:
            # 获取轮廓面积
            area = cv2.contourArea(cnt)
            ####################### 轨迹跟踪
            # 如果轮廓面积大于预设值则跟踪轨迹并显示边框
            if area > areaTH:
                # 计算轮廓的中心点
                m = cv2.moments(cnt)
                cx = int(m['m10'] / m['m00'])
                cy = int(m['m01'] / m['m00'])
                # 计算轮廓的最小矩形边框
                x, y, w, h = cv2.boundingRect(cnt)
                # 新老车辆标识说
                new = True
                # 轮廓y在上下线之间则跟踪
                if cy in range(line_limit_up, line_limit_down) \
                        and cx in range(line_limit_left, line_limit_right):
                    # 将轮廓的矩阵遍历与所有车辆坐标比对，判断轮廓属于哪辆车
                    for i in cars:
                        # 逻辑一：老车辆未过计数线
                        # 当前矩形左上点x-上帧轮廓中心x<=矩阵w，同理y小于h，认为是同一车辆
                        if abs(x - i.getX()) <= w and abs(y - i.getY()) <= h:
                            new = False
                            # 更新坐标
                            i.updateCoords(cx, cy)
                            # 判断是否过下行计数线
                            if i.going_DOWN(line_count_go_down) == True:
                                if isInsidePolygon([i.tracks[-1][0], i.tracks[-1][1]], cedao_pts_1) == True:
                                    cnt_go_down_1 += 1
                                elif isInsidePolygon([i.tracks[-1][0], i.tracks[-1][1]], cedao_pts_2) == True:
                                    cnt_go_down_2 += 1
                                elif isInsidePolygon([i.tracks[-1][0], i.tracks[-1][1]], cedao_pts_3) == True:
                                    cnt_go_down_3 += 1
                                elif isInsidePolygon([i.tracks[-1][0], i.tracks[-1][1]], cedao_pts_4) == True:
                                    cnt_go_down_4 += 1
                                cnt_go_down_all = cnt_go_down_1 + cnt_go_down_2 + cnt_go_down_3 + cnt_go_down_4;
                                print("ID:", i.getId(), 'crossed going down at', time.strftime("%c"))
                            break
                        # 逻辑二：老车辆已过跟踪线：设置done=true
                        if i.getState() == '1':
                            if i.getDir() == 'down' and i.getY() > line_limit_down:
                                i.setDone()
                        # 逻辑三：超时车辆：剔除车辆
                        if i.timedOut():
                            index = cars.index(i)
                            cars.pop(index)
                            del i
                    # 逻辑四：新车辆，加入队列
                    if new == True:  # If nothing is detected,create new
                        p = Car(pid, cx, cy, max_p_age)
                        cars.append(p)
                        pid += 1
                    # 画车辆轮廓中心点：轮廓面积大于预设值并且在跟踪线内
                    cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
                    # 画车辆轮廓矩阵框，轮廓面积大于预设值并且在跟踪线内
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (rectangle_color), 2)
        # 画车辆轮廓中心ID，放在减少绘图次数
        for i in cars:
            cv2.putText(frame, str(i.getId()), (i.getX(), i.getY()), font, 0.5, i.getRGB(), 1, cv2.LINE_AA)
        # 划线
        cv2.polylines(frame, [pts_L1], False, line_count_go_down_color, thickness=2)
        ####################### 测试
        _cedao_pts_1 = cedao_pts_1.reshape((-1, 1, 2))
        cv2.polylines(frame, [_cedao_pts_1], True, (0, 255, 255), thickness=2)
        _cedao_pts_2 = cedao_pts_2.reshape((-1, 1, 2))
        cv2.polylines(frame, [_cedao_pts_2], True, (255, 0, 0), thickness=2)
        _cedao_pts_3 = cedao_pts_3.reshape((-1, 1, 2))
        cv2.polylines(frame, [_cedao_pts_3], True, (204, 0, 255), thickness=2)
        _cedao_pts_4 = cedao_pts_4.reshape((-1, 1, 2))
        cv2.polylines(frame, [_cedao_pts_4], True, (0, 0, 255), thickness=2)
        ####################### 测试
        # 计数器文字
        str_frame = 'frame: ' + str(cnt_frame)
        cv2.putText(frame, str_frame, (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_frame, (10, 20), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        str_go_down_1 = 'goDown1: ' + str(cnt_go_down_1)
        cv2.putText(frame, str_go_down_1, (10, 40), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_go_down_1, (10, 40), font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        str_go_down_2 = 'goDown2: ' + str(cnt_go_down_2)
        cv2.putText(frame, str_go_down_2, (10, 60), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_go_down_2, (10, 60), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        str_go_down_3 = 'goDown3: ' + str(cnt_go_down_3)
        cv2.putText(frame, str_go_down_3, (10, 80), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_go_down_3, (10, 80), font, 0.5, (204, 0, 255), 1, cv2.LINE_AA)
        str_go_down_4 = 'goDown4: ' + str(cnt_go_down_4)
        cv2.putText(frame, str_go_down_4, (10, 100), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_go_down_4, (10, 100), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        str_go_down_all = 'goDownAll: ' + str(cnt_go_down_all)
        cv2.putText(frame, str_go_down_all, (10, 120), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, str_go_down_all, (10, 120), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        # 输出每一帧
        cv2.imshow('Frame', frame)
        # 键盘输入
        if cv2.waitKey(1) & 0xff == ord('w'):
            while (True):
                if cv2.waitKey(1) & 0xff == ord('w'):
                    break
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    # 帧读取失败跳出
    else:
        break
# 关闭释放
cap.release()
cv2.destroyAllWindows()